## ***RNN using PyTorch***

In [1]:
import pandas as pd
df = pd.read_csv("100_Unique_QA_Dataset.csv")

In [2]:
df.sample(3)

,question,answer
26,Who discovered penicillin?,Alexander-Fleming
46,Which bird is known for its ability to mimic s...,Parrot
68,Which sea creature has eight arms?,Octopus


In [3]:
## tokenization
def tokenize(text):
    text = text.lower()
    text = text.replace("?", "")
    text = text.replace("'", "")
    return text.split()


In [4]:
## vocab
vocab = {
    "<UNK>": 0
}

In [5]:
def build_vocab(row):
    tok_ques = tokenize(row["question"])
    tok_ans = tokenize(row["answer"])

    merged_token = tok_ques + tok_ans
    for token in merged_token:
        if token not in vocab:
            vocab[token] = len(vocab)
    # print(merged_token)

In [6]:
## convert words to numerical
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [7]:
len(vocab)

324

## **Text to Indexes**

In [8]:
def text_to_index(text, vocab):
    indexed_text = []
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else: 
            indexed_text.append(vocab["<UNK>"])
    return indexed_text
        

In [9]:
text_to_index("what is the capital of the france", vocab)

[1, 2, 3, 4, 5, 3, 6]

## **Build Custom Dataset**

In [15]:
from torch.utils.data import Dataset, DataLoader
import torch
class QADataset(Dataset):
    def __init__(self, df, vocab):
        super().__init__()
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        numerical_question = text_to_index(self.df.iloc[index]["question"], self.vocab)
        numerical_answer = text_to_index(self.df.iloc[index]["answer"], self.vocab)

        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [16]:
dataset = QADataset(df, vocab)

In [12]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [17]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [18]:
for question, ans in dataloader:
    print(question, ans)

tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([[179]])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([[225]])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([[7]])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([[128]])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([[254]])
tensor([[  1,   2,   3, 141, 117,  83,   3, 277, 278]]) tensor([[121]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([[173]])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([[321]])
tensor([[ 10,   2,  62,  63,   3, 283,   5, 284]]) tensor([[285]])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([[268]])
tensor([[ 42, 137, 118,   3, 247,   5, 248]]) tensor([[249]])
tensor([[42, 86, 87, 88, 89, 39, 90]]) tensor([[91]])
tensor([[ 42,   2,   3, 274, 211, 275]]) tensor([[276]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor(

In [30]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [33]:
learning_rate = 0.001
epochs = 50


In [28]:
model = MySimpleRNN(len(vocab))

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 13.214156
Epoch: 2, Loss: 8.033299
Epoch: 3, Loss: 5.861472
Epoch: 4, Loss: 4.452330
Epoch: 5, Loss: 3.438008
Epoch: 6, Loss: 2.737243
Epoch: 7, Loss: 2.247586
Epoch: 8, Loss: 1.871109
Epoch: 9, Loss: 1.585633
Epoch: 10, Loss: 1.361648
Epoch: 11, Loss: 1.182876
Epoch: 12, Loss: 1.030466
Epoch: 13, Loss: 0.909487
Epoch: 14, Loss: 0.806477
Epoch: 15, Loss: 0.721072
Epoch: 16, Loss: 0.647593
Epoch: 17, Loss: 0.584644
Epoch: 18, Loss: 0.530762
Epoch: 19, Loss: 0.482753
Epoch: 20, Loss: 0.440604
Epoch: 21, Loss: 0.404118
Epoch: 22, Loss: 0.371548
Epoch: 23, Loss: 0.341326
Epoch: 24, Loss: 0.315755
Epoch: 25, Loss: 0.291667
Epoch: 26, Loss: 0.271181
Epoch: 27, Loss: 0.251649
Epoch: 28, Loss: 0.234392
Epoch: 29, Loss: 0.218223
Epoch: 30, Loss: 0.203594
Epoch: 31, Loss: 0.190334
Epoch: 32, Loss: 0.178236
Epoch: 33, Loss: 0.167207
Epoch: 34, Loss: 0.156478
Epoch: 35, Loss: 0.147270
Epoch: 36, Loss: 0.138206
Epoch: 37, Loss: 0.130171
Epoch: 38, Loss: 0.122478
Epoch: 39, Loss: 0.1

In [39]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_index(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [38]:
predict(model, "What is the largest planet in our solar system?")

NameError: name 'text_to_indices' is not defined